In [1]:
import numpy as np
import cmath
import pandas as pd

In [2]:
def phi1_func(g, a, x0):
    if g <= 0:
        mu = np.sqrt(-g)
        A = mu*np.cosh(mu) + np.sinh(mu) - a*x0*np.sinh(mu*x0)
        B = 2.0*mu*np.cosh(mu*x0)
    else:
        mu = np.sqrt(g)
        A = mu*np.cos(mu) + np.sin(mu) - a*x0*np.sin(mu*x0)
        B = 2.0*mu*np.cos(mu*x0)
    return B/A

In [3]:
def phi2_func(mu, a, x0):
    phi = -(2.0*mu*np.cosh(mu*x0))/(np.sinh(mu) + mu*np.cosh(mu) - a*x0*np.sinh(mu*x0))
    return phi.real

In [4]:
def G(y, a, mu, x0):
    return (a*np.cosh(y*x0)-y*np.sinh(y))/(y*y-mu*mu)

In [5]:
def d11_func(g, a, x0):
    if g <= 0:
        mu = np.sqrt(-g)
        A = 16.0*(mu*np.cosh(mu) + np.sinh(mu) - a*x0*np.sinh(mu*x0))
        B = 3.0*g*np.sinh(3.0*mu) + a*mu*np.cosh(3.0*mu*x0)
    else:
        mu = np.sqrt(g)
        A = 16.0*(mu*np.cos(mu) + np.sin(mu) - a*x0*np.sin(mu*x0))
        B = 3.0*g*np.sin(3.0*mu) + a*mu*np.cos(3.0*mu*x0)
    return -B/A-0.75

In [6]:
def d12_func(a, m, k1, k2, n1, n2, x0):
    B = 3.0*m*(G(k1, a, m, x0) + G(k2, a, m, x0) + G(n1, a, m, x0) + G(n2, a, m, x0))
    A = np.sinh(m)+m*np.cosh(m)-a*x0*np.sinh(m*x0)
    d = B/A
    return d.real

In [7]:
def d21_func(a, m, k1, k2, x0):
    B = 3.0*m*(G(k1, a, m, x0) + G(k2, a, m, x0))
    A = 2.0*(np.sinh(m)+m*np.cosh(m)-a*x0*np.sinh(m*x0))
    d = B/A
    return d.real-1.5

In [8]:
def d22_func(a, mu, kappa, nu, x0):
    mu_conj = np.conj(mu)
    B = 3.0*mu*(G(kappa, a, mu, x0) + G(nu, a, mu, x0) + 2.0*G(mu_conj, a, mu, x0))
    A = 2.0*(np.sinh(mu)+mu*np.cosh(mu)-a*x0*np.sinh(mu*x0))
    d = B/A
    return d.real

In [9]:
def get_alpha_u(g, x0):
    if g <= 0:
        mu = np.sqrt(-g)
        return mu*np.sinh(mu)/np.cosh(mu*x0)
    else:
        mu = np.sqrt(g)
        return -mu*np.sin(mu)/np.cos(mu*x0)

In [10]:
g = [5.03033, 5.18779, 5.3747, 5.60133, 5.8849, 6.2581, 6.7976, 7.7936]
w_f = [0.069, 0.013, 0.009, 0.032, 0.039, 0.027, 0.02, 0.02]
a_f = [-2.481, -2.6028, -2.75, -2.9307, -3.161, -3.47, -3.922, -4.74999]
x0 = np.linspace(0.35, 0.49, 8)
a_0 = [get_alpha_u(g[idx], x0[idx]) for idx in range(len(x0))]
i = 1.0j

In [11]:
a_0

[-2.481135323832114,
 -2.602757811110056,
 -2.749535427816128,
 -2.9306153226898792,
 -3.1612541662521148,
 -3.4700034263053077,
 -3.9218995496154214,
 -4.749906523717664]

In [12]:
cols = ['x0', 'delta', 'delta1', 'delta2', 'rho1', 'rho2', 'Stability', 'phi1', 'phi2', 'd11', 'd12', 'd21', 'd22']
df = pd.DataFrame(columns=cols)
df.head()

,x0,delta,delta1,delta2,rho1,rho2,Stability,phi1,phi2,d11,d12,d21,d22


In [13]:
for idx in range(len(x0)):
    if g[idx] <= 0:
        mu_0 = np.sqrt(-g[idx])
    else:
        mu_0 = np.sqrt(g[idx])
    mu_f = np.sqrt(-g[idx] + i*w_f[idx])
    kappa_f = mu_f + 2.0*mu_f.real
    nu_f = mu_f + 2.0*i*mu_f.imag
    mu_f_conj = np.conj(mu_f)
    kappa11 = mu_0 + 2.0*mu_f.real
    kappa12 = mu_0 - 2.0*mu_f.real
    nu11 = mu_0 + 2.0*i*mu_f.imag
    nu12 = mu_0 - 2.0*i*mu_f.imag
    kappa21 = mu_f + 2.0*mu_0
    kappa22 = mu_f - 2.0*mu_0
    phi1 = abs(phi1_func(g[idx], a_0[idx], x0[idx]))
    phi2 = abs(phi2_func(mu_f, a_f[idx], x0[idx]))
    d11 = d11_func(g[idx], a_0[idx], x0[idx])
    d12 = d12_func(a_0[idx], mu_0, kappa11, kappa12, nu11, nu12, x0[idx])
    d21 = d21_func(a_f[idx], mu_f, kappa21, kappa22, x0[idx])
    d22 = d22_func(a_f[idx], mu_f, kappa_f, nu_f, x0[idx])
    delta = d11*d22 - d12*d21
    delta1 = -phi1*d22+phi2*d12
    delta2 = -phi2*d11+phi1*d21
    # bad solution
    if delta > 0.0:
        delta1, delta2 = abs(delta1), abs(delta2)
        stab = 'Stable'
    else:
        delta1, delta2 = -abs(delta1), -abs(delta2)
        stab = 'Unstable'
    #
    rho1, rho2 = np.sqrt(delta1/delta), np.sqrt(delta2/delta)
    df.loc[idx] =[x0[idx], delta, delta1, delta2, rho1, rho2, stab, phi1, phi2, d11, d12, d21, d22]

In [14]:
df.head(10)

,x0,delta,delta1,delta2,rho1,rho2,Stability,phi1,phi2,d11,d12,d21,d22
0,0.35,-7.331671e+04,-3.566818e+05,-7.508634e+06,2.205662,10.119964,Unstable,3.569718e+05,0.226814,7.333893e+04,-1.776921,-20.987648,-0.999189
1,0.37,-1.400484e+06,-5.232724e+06,-6.142481e+07,1.932969,6.622665,Unstable,3.261731e+06,0.421975,8.729472e+05,-1.810524,-18.719034,-1.604279
2,0.39,-1.613592e+07,-4.675355e+07,-2.799201e+08,1.702200,4.165050,Unstable,3.361144e+06,13.699673,1.160033e+06,-1.851710,88.009378,-13.910021
3,0.41,-9.229812e+04,-2.077236e+05,-4.240477e+06,1.500191,6.778147,Unstable,1.679491e+05,0.158632,7.458624e+04,-1.902605,-25.178135,-1.236826
4,0.43,-2.763304e+04,-4.793461e+04,-2.868497e+06,1.317074,10.188561,Unstable,7.407990e+04,0.452388,4.258740e+04,-1.967113,-38.461592,-0.647078
5,0.45,-7.668747e+04,-1.012243e+05,-3.855970e+06,1.148895,7.090954,Unstable,9.798770e+04,0.053390,7.415725e+04,-2.052581,-39.311168,-1.033032
6,0.47,-1.747595e+05,-1.691139e+05,-2.812161e+06,0.983715,4.011434,Unstable,1.004976e+05,0.853975,1.038164e+05,-2.175728,-27.100196,-1.682784
7,0.49,-8.773305e+04,-5.659634e+04,-2.029587e+06,0.803179,4.809748,Unstable,3.625367e+04,0.826312,5.611291e+04,-2.394436,-54.703997,-1.561175
